In [67]:
pip install flask_sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [127]:
#load libraries

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta

In [129]:
#configuration of the app
app = Flask('COP1_du_13')
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = 'supersecretkey'
db = SQLAlchemy(app)

In [115]:
# Baseline for "volunteer=Benevole"
class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    prenom = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    telephone = db.Column(db.String(20), nullable=False)

In [71]:
# load the excel
file_path = '~/Desktop/cop1_app/benevole.xlsx/benevole.xlsx' 
#read the excel 
df = pd.read_excel(file_path)
#look at the first lines
print(df.head())

               Horodateur      Prénom        Nom     Date de naissance  \
0 2023-09-10 17:07:48.863      Axelle      Renaud  2001-09-02 00:00:00   
1 2023-09-10 17:14:41.500        Asma  Ikhenazene  1998-07-20 00:00:00   
2 2023-09-10 17:14:54.145    Cielinda       Sejil  2002-03-13 00:00:00   
3 2023-09-10 17:23:38.388  Bouthaina    Hasnaoui   1995-04-17 00:00:00   
4 2023-09-10 17:25:54.502   Zeinabou   Hainikoye   2004-12-18 00:00:00   

  Cursus suivi et niveau d'études pour l'année 2024-2025  \
0                                   Master économie        
1                                   M2 neurosciences       
2                             2 e année de médecine        
3                                           FARC TEC       
4                                Économie gestion L2       

                 Université / établissement Numéro de Téléphone  \
0                                       AMU          0640675168   
1                                      AMU        +213666014

In [76]:
#i will not use all the columns
#using the most interesting 
df_benevoles = df[['Prénom', 'Nom ', 'Numéro de Téléphone', 'Ton adresse mail ?']]

In [79]:
#rename them to fit the principal code i've wrote
df_benevoles.columns = ['prenom', 'nom', 'telephone', 'email']

In [97]:
from sqlalchemy import or_

def importer_benevoles():
    for _, row in df_benevoles.iterrows():
        # Vérification des valeurs essentielles (nom, prénom, email ou téléphone requis)
        if pd.notna(row['nom']) and pd.notna(row['prenom']):
            nom = str(row['nom']).strip()
            prenom = str(row['prenom']).strip()
            email = str(row['email']).strip() if pd.notna(row['email']) else None
            telephone = None

            if pd.notna(row['telephone']):  
                try:
                    telephone = str(int(row['telephone']))  # Convertir en entier puis en str
                except ValueError:
                    telephone = None  # Si conversion impossible, on met None

            # Si aucun téléphone, on passe à la ligne suivante
            if not telephone:
                continue  

            # Vérifier si l'email ou le téléphone existent déjà
            conditions = []
            if email:
                conditions.append(Benevole.email == email)
            if telephone:
                conditions.append(Benevole.telephone == telephone)

            exists = Benevole.query.filter(or_(*conditions)).first() if conditions else None

            if not exists:
                benevole = Benevole(
                    prenom=prenom,
                    nom=nom,
                    email=email,
                    telephone=telephone
                )
                db.session.add(benevole)  # Ajout du bénévole à la session

    db.session.commit()  # Validation des ajouts dans la base de données


In [99]:
#call the importation function when the apllication start
if __name__ == '__main__':
    with app.app_context():
        db.create_all()  #create table if it's do not exists
        importer_benevoles() 
    app.run(debug=True)

 * Serving Flask app 'COP1_du_13'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [101]:
# Baseline for the "task= mission"
class Mission(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    titre = db.Column(db.String(100), nullable=False)
    date = db.Column(db.String(20), nullable=False)
    heure = db.Column(db.String(10), nullable=False)
    nb_places = db.Column(db.Integer, nullable=False)

In [95]:
# Baseline of inscriptions (volunteer register for tasks)
class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'), nullable=False)
    mission_id = db.Column(db.Integer, db.ForeignKey('mission.id'), nullable=False)
    date_inscription = db.Column(db.DateTime, default=datetime.utcnow)

/opt/anaconda3/lib/python3.12/site-packages/flask_sqlalchemy/model.py:144: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Inscription, and will be replaced in the string-lookup table.
  super().__init__(name, bases, d, **kwargs)


InvalidRequestError: Table 'inscription' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [173]:
#path for the index
@app.route('/')
def indexe():
    return render_template('index.html')

AssertionError: View function mapping is overwriting an existing endpoint function: indexe

In [171]:
#path for the volunteer
@app.route('/benevoles')
def benevoles():
    benevoles = Benevole.query.all()
    return render_template('benevoles.html', benevoles=benevoles)

AssertionError: View function mapping is overwriting an existing endpoint function: benevoles

In [169]:
@app.route('/missions')
def missions():
    missions = Mission.query.all()
    benevoles = Benevole.query.all()
    return render_template('mission.html', missions=missions, benevoles=benevoles)

@app.route('/inscrire', methods=['POST'])
def inscrire():
    benevole_id = request.form['benevole_id']
    mission_id = request.form['mission_id']

    mission = Mission.query.get(mission_id)
    if mission and mission.nb_places > 0:
        inscription = Inscription(benevole_id=benevole_id, mission_id=mission_id)
        db.session.add(inscription)
        mission.nb_places -= 1
        db.session.commit()
        flash('✅ Inscription réussie !')
    else:
        flash('❌ Plus de place disponible pour cette mission.')

    return redirect(url_for('missions'))


AssertionError: View function mapping is overwriting an existing endpoint function: missions

In [175]:
# Route pour afficher le classement des bénévoles
@app.route('/classement')
def classement():
    classement = db.session.query(
        Benevole.nom, Benevole.prenom, db.func.count(Inscription.id).label('nb_missions')
    ).join(Inscription).group_by(Benevole.id).order_by(db.desc('nb_missions')).all()

    return render_template('classement.html', classement=classement)

In [207]:
# Fonction pour envoyer une notification aux bénévoles inactifs
def notifier_inactifs():
    seuil = datetime.utcnow() - timedelta(days=60)
    inactifs = db.session.query(Benevole).outerjoin(Inscription, Benevole.id == Inscription.benevole_id) \
        .filter((Inscription.date_inscription == None) | (Inscription.date_inscription < seuil)).all()

    for benevole in inactifs:
        envoyer_email(benevole.email, "🌟 Rappel : Engagez-vous dans une mission !",
                      f"Bonjour {benevole.prenom}, nous avons remarqué que vous n'avez pas participé à une mission récemment. "
                      "Rejoignez une mission dès maintenant !")

In [209]:
# Fonction d'envoi d'email
def envoyer_email(destinataire, sujet, message):
    expediteur = "tonemail@gmail.com"
    mot_de_passe = "motdepasse"
    msg = MIMEText(message)
    msg["Subject"] = sujet
    msg["From"] = expediteur
    msg["To"] = destinataire

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(expediteur, mot_de_passe)
        server.sendmail(expediteur, destinataire, msg.as_string())
        server.quit()
        print(f"📧 Email envoyé à {destinataire}")
    except Exception as e:
        print(f"⚠️ Erreur lors de l'envoi de l'email : {e}")

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True)


 * Serving Flask app 'COP1_du_13'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
